In [3]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

model_name = 'distilbert-base-uncased'
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)  # 6 emotions
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [32]:
def preprocess_text(text):
    encodings = tokenizer(text, padding=True, truncation=True, return_tensors="tf")
    return encodings

In [19]:
emotion_labels = ["happy", "sad", "angry", "anxious", "neutral", "excited"]

In [33]:
def predict_emotion(text):
    inputs=preprocess_text(text)
    outputs=model(inputs)
    logits=outputs.logits
    predicted_class=tf.argmax(logits, axis=1)
    return emotion_labels[predicted_class.numpy()[0]]

In [34]:
def generate_response(emotion):
    responses = {
        "happy": "I'm glad you're feeling good! 😊 What's making you so happy?",
        "sad": "I'm sorry to hear that. Want to talk about what's on your mind? 💙",
        "angry": "I understand that you're upset. Take a deep breath. 😌",
        "anxious": "It's okay to feel anxious. Would you like to share more? 💭",
        "neutral": "I see you're feeling neutral. What's going on? 🤔",
        "excited": "That’s awesome! Tell me more about what’s got you excited! 🎉"
    }

    return responses.get(emotion, "I’m here to listen! 😊")

# Example interaction
user_input = "I'm feeling a bit overwhelmed today."
predicted_emotion = predict_emotion(user_input)
response = generate_response(predicted_emotion)
print(f"Emotion: {predicted_emotion}")
print(f"Response: {response}")

Emotion: excited
Response: That’s awesome! Tell me more about what’s got you excited! 🎉


In [36]:

import tensorflow as tf

model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    x= preprocess_text(user_input) ,  # Pass the tokenized training data
    y=emotion_labels,      # Pass the training labels as a TensorFlow tensor
    epochs=3,            # Adjust as needed
    batch_size=16
)